# Лабораторная работа № 1 Задача регрессии

## Задание
Решите пожалуйста следующие задачи.

1. Примените следующие типы регрессий для датасета представленного по
[ссылке](https://drive.google.com/file/d/1LBDnhITL0Wqwp5G6M6IBI-SSz8BIoNec/view?usp=share_link).
- Линейная регрессия
- LASSO регрессия
- Ridge регрессия
- ElasticNet регрессия
- Polynom регрессия (для степени полинома 2)
- Polynom регрессия (для степени полинома 5)
- Polynom регрессия (для степени полинома 10)

Постройте модель для атрибутов `X = {SkinThickness, BMI}` и `Y = {Insulin}`.  
**Укажите `score` (r2_score) моделей. Найдите метрики MAE, MSE.**


## Загрузка данных

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
! gdown --fuzzy 'https://drive.google.com/file/d/1LBDnhITL0Wqwp5G6M6IBI-SSz8BIoNec/view?usp=share_link'

Downloading...
From: https://drive.google.com/uc?id=1LBDnhITL0Wqwp5G6M6IBI-SSz8BIoNec
To: /content/diabetes.csv
100% 23.1k/23.1k [00:00<00:00, 40.1MB/s]


In [ ]:
df = pd.read_csv('/content/diabetes.csv')
df

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1
...,...,...,...,...,...,...,...,...,...
763,10,101,76,48,180,32.9,0.171,63,0
764,2,122,70,27,0,36.8,0.340,27,0
765,5,121,72,23,112,26.2,0.245,30,0
766,1,126,60,0,0,30.1,0.349,47,1


##  О датасете
[Pima Indians Diabetes Database](https://www.kaggle.com/datasets/uciml/pima-indians-diabetes-database/data)

Этот набор данных взят из Национального института диабета, заболеваний органов пищеварения и почек (NIDDK). Целью набора данных является диагностическое предсказание наличия у пациента диабета на основе определенных диагностических измерений, включенных в набор данных. На выбор этих экземпляров из более крупной базы данных было наложено несколько ограничений. В частности, все пациенты здесь — женщины не моложе 21 года, родом из индейцев Пима.

Наборы данных состоят из нескольких переменных медицинских предикторов и одной целевой переменной `Outcome`. Переменные-предикторы включают количество беременностей, которые были у пациентки, её индекс массы тела `BMI`, уровень инсулина, возраст и так далее. Всего в датасете 9 переменных:
- `Pregnancies` - количество беременностей;
- `Glucose` - концентрация глюкозы в плазме через 2 часа при пероральном тесте на толерантность к глюкозе;
- `BloodPressure` - диастолическое артериальное давление (мм рт. ст.);
- `SkinThickness` - толщина кожной складки трицепса (мм);
- `Insulin` - 2-часовой сывороточный инсулин (мкЕд/мл);
- `BMI` - Индекс массы тела (вес в кг/(рост в м)^2);
- `DiabetesPedigreeFunction` - функция оценки вероятности диабета на основе семейного анамнеза;
- `Age` - возраст (лет);
- `Outcome` - целевая переменная (0 или 1) 268 из 768 — 1 (больны), остальные — 0 (здоровы).



In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 768 entries, 0 to 767
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Pregnancies               768 non-null    int64  
 1   Glucose                   768 non-null    int64  
 2   BloodPressure             768 non-null    int64  
 3   SkinThickness             768 non-null    int64  
 4   Insulin                   768 non-null    int64  
 5   BMI                       768 non-null    float64
 6   DiabetesPedigreeFunction  768 non-null    float64
 7   Age                       768 non-null    int64  
 8   Outcome                   768 non-null    int64  
dtypes: float64(2), int64(7)
memory usage: 54.1 KB


In [ ]:
df.describe()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
count,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000
mean,3.845052,120.894531,69.105469,20.536458,79.799479,31.992578,0.471876,33.240885,0.348958
std,3.369578,31.972618,19.355807,15.952218,115.244002,7.884160,0.331329,11.760232,0.476951
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.078000,21.000000,0.000000
25%,1.000000,99.000000,62.000000,0.000000,0.000000,27.300000,0.243750,24.000000,0.000000
50%,3.000000,117.000000,72.000000,23.000000,30.500000,32.000000,0.372500,29.000000,0.000000
75%,6.000000,140.250000,80.000000,32.000000,127.250000,36.600000,0.626250,41.000000,1.000000
max,17.000000,199.000000,122.000000,99.000000,846.000000,67.100000,2.420000,81.000000,1.000000


In [ ]:
df.columns

Index(['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin',
       'BMI', 'DiabetesPedigreeFunction', 'Age', 'Outcome'],
      dtype='object')

## Линейная регрессия

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

X = df[['SkinThickness', 'BMI']]
Y = df['Insulin']

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

linear_reg = LinearRegression()
linear_reg.fit(X_train, Y_train)

Y_pred_linear = linear_reg.predict(X_test)

r2_linear = r2_score(Y_test, Y_pred_linear)
mae_linear = mean_absolute_error(Y_test, Y_pred_linear)
mse_linear = mean_squared_error(Y_test, Y_pred_linear)

print(f"Linear Regression R2 Score: {r2_linear:.4f}")
print(f"Linear Regression MAE: {mae_linear:.4f}")
print(f"Linear Regression MSE: {mse_linear:.4f}")

Linear Regression R2 Score: 0.1812
Linear Regression MAE: 67.2310
Linear Regression MSE: 10083.7350


## LASSO регрессия

In [ ]:
from sklearn.linear_model import Lasso

lasso_reg = Lasso(alpha=1.0, random_state=42)
lasso_reg.fit(X_train, Y_train)

Y_pred_lasso = lasso_reg.predict(X_test)

r2_lasso = r2_score(Y_test, Y_pred_lasso)
mae_lasso = mean_absolute_error(Y_test, Y_pred_lasso)
mse_lasso = mean_squared_error(Y_test, Y_pred_lasso)

print(f"LASSO Regression R2 Score: {r2_lasso:.4f}")
print(f"LASSO Regression MAE: {mae_lasso:.4f}")
print(f"LASSO Regression MSE: {mse_lasso:.4f}")

LASSO Regression R2 Score: 0.1813
LASSO Regression MAE: 67.2542
LASSO Regression MSE: 10082.0923


## Ridge регрессия

In [ ]:
from sklearn.linear_model import Ridge

ridge_reg = Ridge(alpha=1.0, random_state=42)
ridge_reg.fit(X_train, Y_train)

Y_pred_ridge = ridge_reg.predict(X_test)

r2_ridge = r2_score(Y_test, Y_pred_ridge)
mae_ridge = mean_absolute_error(Y_test, Y_pred_ridge)
mse_ridge = mean_squared_error(Y_test, Y_pred_ridge)

print(f"Ridge Regression R2 Score: {r2_ridge:.4f}")
print(f"Ridge Regression MAE: {mae_ridge:.4f}")
print(f"Ridge Regression MSE: {mse_ridge:.4f}")

Ridge Regression R2 Score: 0.1812
Ridge Regression MAE: 67.2311
Ridge Regression MSE: 10083.7272


## ElasticNet регрессия

In [ ]:
from sklearn.linear_model import ElasticNet

elastic_net_reg = ElasticNet(random_state=42)
elastic_net_reg.fit(X_train, Y_train)

Y_pred_elastic_net = elastic_net_reg.predict(X_test)

r2_elastic_net = r2_score(Y_test, Y_pred_elastic_net)
mae_elastic_net = mean_absolute_error(Y_test, Y_pred_elastic_net)
mse_elastic_net = mean_squared_error(Y_test, Y_pred_elastic_net)

print(f"ElasticNet Regression R2 Score: {r2_elastic_net:.4f}")
print(f"ElasticNet Regression MAE: {mae_elastic_net:.4f}")
print(f"ElasticNet Regression MSE: {mse_elastic_net:.4f}")

ElasticNet Regression R2 Score: 0.1814
ElasticNet Regression MAE: 67.2802
ElasticNet Regression MSE: 10080.5488


## Polynom регрессия (для степени полинома 2)

In [ ]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

poly_features_2 = PolynomialFeatures(degree=2)
X_train_poly_2 = poly_features_2.fit_transform(X_train)
X_test_poly_2 = poly_features_2.transform(X_test)

poly_reg_2 = LinearRegression()
poly_reg_2.fit(X_train_poly_2, Y_train)

Y_pred_poly_2 = poly_reg_2.predict(X_test_poly_2)

r2_poly_2 = r2_score(Y_test, Y_pred_poly_2)
mae_poly_2 = mean_absolute_error(Y_test, Y_pred_poly_2)
mse_poly_2 = mean_squared_error(Y_test, Y_pred_poly_2)

print(f"Polynomial Regression (Degree 2) R2 Score: {r2_poly_2:.4f}")
print(f"Polynomial Regression (Degree 2) MAE: {mae_poly_2:.4f}")
print(f"Polynomial Regression (Degree 2) MSE: {mse_poly_2:.4f}")

Polynomial Regression (Degree 2) R2 Score: 0.1680
Polynomial Regression (Degree 2) MAE: 66.2456
Polynomial Regression (Degree 2) MSE: 10245.4591


## Polynom регрессия (для степени полинома 5)

In [ ]:
poly_features_5 = PolynomialFeatures(degree=5)
X_train_poly_5 = poly_features_5.fit_transform(X_train)
X_test_poly_5 = poly_features_5.transform(X_test)

poly_reg_5 = LinearRegression()
poly_reg_5.fit(X_train_poly_5, Y_train)

Y_pred_poly_5 = poly_reg_5.predict(X_test_poly_5)

r2_poly_5 = r2_score(Y_test, Y_pred_poly_5)
mae_poly_5 = mean_absolute_error(Y_test, Y_pred_poly_5)
mse_poly_5 = mean_squared_error(Y_test, Y_pred_poly_5)

print(f"Polynomial Regression (Degree 5) R2 Score: {r2_poly_5:.4f}")
print(f"Polynomial Regression (Degree 5) MAE: {mae_poly_5:.4f}")
print(f"Polynomial Regression (Degree 5) MSE: {mse_poly_5:.4f}")

Polynomial Regression (Degree 5) R2 Score: -102.1129
Polynomial Regression (Degree 5) MAE: 153.2557
Polynomial Regression (Degree 5) MSE: 1269818.1114


## Polynom регрессия (для степени полинома 10)

In [ ]:
poly_features_10 = PolynomialFeatures(degree=10)
X_train_poly_10 = poly_features_10.fit_transform(X_train)
X_test_poly_10 = poly_features_10.transform(X_test)

poly_reg_10 = LinearRegression()
poly_reg_10.fit(X_train_poly_10, Y_train)

Y_pred_poly_10 = poly_reg_10.predict(X_test_poly_10)

r2_poly_10 = r2_score(Y_test, Y_pred_poly_10)
mae_poly_10 = mean_absolute_error(Y_test, Y_pred_poly_10)
mse_poly_10 = mean_squared_error(Y_test, Y_pred_poly_10)

print(f"Polynomial Regression (Degree 10) R2 Score: {r2_poly_10:.4f}")
print(f"Polynomial Regression (Degree 10) MAE: {mae_poly_10:.4f}")
print(f"Polynomial Regression (Degree 10) MSE: {mse_poly_10:.4f}")

Polynomial Regression (Degree 10) R2 Score: -30000512581.1079
Polynomial Regression (Degree 10) MAE: 1548976.1908
Polynomial Regression (Degree 10) MSE: 369451263871929.6250


## Вывод

**Анализ результатов:**

Из полученных метрик видно, что R2 score для линейной регрессии, LASSO, Ridge и ElasticNet регрессий находятся в диапазоне от 0.16 до 0.18, что говорит о том, что эти модели очень плохо объясняют дисперсию целевой переменной (Insulin) на основе выбранных признаков (SkinThickness и BMI). Низкие значения MAE и MSE для этих моделей также указывают на то, что предсказания далеки от фактических значений.

Полиномиальная регрессия со степенью 2 показывает немного худший результат по R2 score, чем линейные модели, но MAE и MSE сопоставимы.

Полиномиальные регрессии со степенями 5 и 10 демонстрируют очень низкие (отрицательные) значения R2 score и крайне высокие значения MAE и MSE. Это свидетельствует о сильном переобучении (overfitting) моделей на тренировочных данных. Модели высокой степени сложности (степень 5 и 10) пытаются слишком точно подогнаться под тренировочные данные, включая шум, что приводит к плохой обобщающей способности на тестовых данных.

**Вывод о применимости:**

На основе данного анализа можно сделать вывод, что ни один из примененных типов регрессии с использованием признаков SkinThickness и BMI не показал хороших результатов для предсказания уровня Insulin. Значения R2 score близки к нулю или отрицательны, что указывает на то, что модели не способны адекватно объяснить зависимость.

В текущем виде, ни одна из примененных моделей не является применимой для точного предсказания уровня Insulin на основе признаков SkinThickness и BMI.